In [ ]:
import os
import pandas as pd

# Dapatkan path ke folder saat ini
folder_path = os.getcwd()

# Filter file yang diawali dengan "spklu_pulau_sumatera" dan berformat .csv
csv_files = [f for f in os.listdir(folder_path) if f.startswith("spklu_pulau_sumatera") and f.endswith(".csv")]

# Gabungkan semua file CSV
combined_df = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files], ignore_index=True)

# Simpan hasil gabungan ke file baru (opsional)
combined_df.to_csv("gabungan_spklu_sumatera.csv", index=False)

print(f"{len(csv_files)} file berhasil digabungkan.")


In [ ]:
# Baca file CSV
df = pd.read_csv("gabungan_spklu_sumatera.csv")

# Ekstrak angka setelah "/" pada kolom 'Konektor' sebagai jumlah total konektor
df['Jumlah Konektor'] = df['Konektor'].str.extract(r'/(\d+)').astype(int)

df['Jumlah Tersedia'] = df['Konektor'].str.extract(r'(\d+)/').astype(int)

# (Opsional) Simpan ke file baru
df.to_csv("gabungan_spklu_sumatera.csv", index=False)

print("Kolom 'Jumlah Konektor' berhasil ditambahkan dan file disimpan sebagai 'gabungan_spklu_sumatera.csv'")


In [ ]:
import pandas as pd

# Baca file
df = pd.read_csv("gabungan_spklu_sumatera.csv")

# Hitung panjang setiap entri
df['Len Charging Slots'] = df['Charging Slots'].apply(lambda x: len(eval(x)) if pd.notnull(x) else 0)
df['Len Ketersediaan'] = df['Ketersediaan'].apply(lambda x: len(eval(x)) if pd.notnull(x) else 0)

# Ambil hanya baris yang valid
clean_df = df[
    (df['Len Charging Slots'] == df['Jumlah Konektor']) &
    (df['Len Ketersediaan'] == df['Jumlah Konektor'])
]

# Simpan ulang ke file yang sama
clean_df.to_csv("gabungan_spklu_sumatera_benar.csv", index=False)

print("Baris tidak valid telah dihapus dan file disimpan ulang sebagai 'gabungan_spklu_sumatera_benar.csv'")


In [ ]:
import pandas as pd

# Baca file
df = pd.read_csv("gabungan_spklu_sumatera.csv")

# Hitung panjang setiap entri
df['Len Charging Slots'] = df['Charging Slots'].apply(lambda x: len(eval(x)) if pd.notnull(x) else 0)
df['Len Ketersediaan'] = df['Ketersediaan'].apply(lambda x: len(eval(x)) if pd.notnull(x) else 0)

# Ambil hanya baris yang valid
false_df = df[
    (df['Len Charging Slots'] != df['Jumlah Konektor']) |
    (df['Len Ketersediaan'] != df['Jumlah Konektor'])
]

# Simpan ulang ke file yang sama
false_df.to_csv("gabungan_spklu_sumatera_salah.csv", index=False)

print("Baris tidak valid telah dihapus dan file disimpan ulang sebagai 'gabungan_spklu_sumatera_salah.csv'")


In [ ]:
import pandas as pd

# Baca file CSV
df = pd.read_csv("gabungan_spklu_sumatera_benar.csv")

# Ubah kolom Charging Slots menjadi list (dari string)
df['Charging Slots List'] = df['Charging Slots'].apply(lambda x: eval(x) if pd.notnull(x) else [])

# Hapus baris yang memiliki 'A CCS2' atau 'B CCS2'
clean_df = df[~df['Charging Slots List'].apply(lambda slots: any(s in ['A CCS2', 'B CCS2'] for s in slots))].copy()

# Hapus kolom bantu
clean_df = clean_df.drop(columns=['Charging Slots List'])

# Simpan hasil ke file baru
clean_df.to_csv("gabungan_spklu_sumatera_cleaned.csv", index=False)

print("Baris dengan 'A CCS2' atau 'B CCS2' telah dihapus dan file disimpan sebagai 'gabungan_spklu_sumatera_cleaned.csv'")


In [ ]:
import pandas as pd

# Baca file CSV hasil pembersihan
df = pd.read_csv("gabungan_spklu_sumatera_cleaned.csv")

# Hitung jumlah nilai unik pada kolom "Nama SPKLU"
jumlah_unik_spklu = df["Nama SPKLU"].nunique()

print(f"Jumlah nilai unik pada kolom 'Nama SPKLU': {jumlah_unik_spklu}")


In [ ]:
import pandas as pd
from collections import defaultdict, Counter

# Baca data
df = pd.read_csv("gabungan_spklu_sumatera_cleaned.csv")

# Konversi kolom Charging Slots dan Ketersediaan dari string ke list/tuple
df['Charging Slots List'] = df['Charging Slots'].apply(lambda x: eval(x) if pd.notnull(x) else [])
df['Ketersediaan List'] = df['Ketersediaan'].apply(lambda x: eval(x) if pd.notnull(x) else [])

# Hasil akhir
result = []

# Kelompokkan berdasarkan Nama SPKLU
for name, group in df.groupby("Nama SPKLU"):
    slot_counter = defaultdict(lambda: [0, 0])  # format: {'22 kW': [tersedia, total]}

    for _, row in group.iterrows():
        slots = row["Charging Slots List"]
        availability = row["Ketersediaan List"]
        for s, a in zip(slots, availability):
            slot_counter[s][1] += 1  # total
            if a == 1:
                slot_counter[s][0] += 1  # tersedia

    # Format akhir seperti {'22 kW': '3/4'}
    formatted = {k: f"{v[0]}/{v[1]}" for k, v in slot_counter.items()}
    
    # Ambil Charging Slots dari baris pertama untuk referensi
    first_charging_slot = group.iloc[0]['Charging Slots']
    slot_list = eval(first_charging_slot)
    
    # Hitung jumlah tiap daya konektor
    number_slot_count = dict(Counter(slot_list))

    result.append({
        "Nama SPKLU": name,
        "Charging Slots": first_charging_slot,
        "Number of Charging Slots": number_slot_count,
        "Ketersediaan History": formatted
    })

# Buat dataframe hasil
history_df = pd.DataFrame(result)

# Simpan ke file CSV
history_df.to_csv("ketersediaan_history_per_spklu.csv", index=False)

print("File disimpan sebagai 'ketersediaan_history_per_spklu.csv'")

In [ ]:
import pandas as pd

# Langkah 1: Baca file CSV
ketersediaan_df = pd.read_csv("ketersediaan_history_per_spklu.csv")
long_lat_df = pd.read_csv("long_lat_spklu_sumatera.csv")

# Langkah 2: Ambil daftar Nama SPKLU yang valid
valid_spklu_names = set(long_lat_df["Nama SPKLU"].dropna().unique())

# Langkah 3: Filter baris yang Nama SPKLU-nya ada di daftar valid
filtered_df = ketersediaan_df[ketersediaan_df["Nama SPKLU"].isin(valid_spklu_names)]

# Langkah 4: Simpan kembali ke file dengan nama yang sama (overwrite)
filtered_df.to_csv("ketersediaan_history_per_spklu.csv", index=False)

# Langkah 5: Cetak jumlah baris setelah difilter
print(f"Jumlah baris setelah difilter: {len(filtered_df)}")

In [ ]:
import pandas as pd
import ast

# Baca file CSV
df = pd.read_csv("ketersediaan_history_per_spklu.csv")

# Konstanta
battery_capacity = 74.01
charging_efficiency = 0.85

# Fungsi parsing dictionary
def parse_dict_column(value):
    try:
        return ast.literal_eval(value)
    except:
        return {}

# Buat kolom Parameter
parameter_list = []

for i, row in df.iterrows():
    charging_dict = parse_dict_column(row["Number of Charging Slots"])
    availability_dict = parse_dict_column(row["Ketersediaan History"])
    
    param_entry = {}
    for rate, count in charging_dict.items():
        try:
            tersedia, total = map(int, availability_dict[rate].split("/"))
            p = 1 - (tersedia / total)
            if p == 0:
                p = 0.025  # perintah khusus
            s = count
            k = s + 1
            rate_kw = float(rate.split()[0])
            service_rate = 1 / ((battery_capacity * charging_efficiency) / rate_kw)
            arrival_rate = p * service_rate
            
            param_entry[rate] = {
                "p": round(p, 4),
                "k": k,
                "service_rate": round(service_rate, 4),
                "arrival_rate": round(arrival_rate, 4),
                "s": s
            }
        except Exception:
            continue

    parameter_list.append(param_entry)

# Tambahkan kolom ke dataframe
df["Parameter"] = parameter_list

# Simpan ke file baru
df.to_csv("ketersediaan_history_with_parameters.csv", index=False)

print("✅ File berhasil disimpan sebagai 'ketersediaan_history_with_parameters.csv'")


In [ ]:
import pandas as pd
import ast

# Baca file hasil sebelumnya
df = pd.read_csv("ketersediaan_history_with_parameters.csv")
df["Parameter"] = df["Parameter"].apply(ast.literal_eval)

# Konstanta
battery_capacity = 74.01
efficiency = 0.85
minute_factor = 60

# Ambil kombinasi unik
unique_combinations = {}

for param_dict in df["Parameter"]:
    for rate, values in param_dict.items():
        key = (rate, tuple(sorted(values.items())))
        if key not in unique_combinations:
            unique_combinations[key] = values

# Buat list hasil bersih
unique_param_list = []

for key, values in unique_combinations.items():
    rate_str = key[0]           # '22 kW'
    rate_int = int(rate_str.split()[0])  # 22

    avg_time = (battery_capacity * efficiency * minute_factor) / rate_int

    entry = {
        "Charging Rate": rate_int,
        **dict(values),
        "average_charging_time": round(avg_time, 2)
    }

    unique_param_list.append(entry)

# Simpan ke DataFrame
unique_df = pd.DataFrame(unique_param_list)
unique_df.to_csv("unique_charging_rate_parameters_cleaned.csv", index=False)

print("✅ File disimpan sebagai 'unique_charging_rate_parameters_cleaned.csv'")
